In [2]:

import pyvo as vo
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.table import Table
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils.utils import calc_total_error
import pandas as pd
from scipy.spatial import KDTree
import json


from scipy.optimize import curve_fit
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std, sigma_clipped_stats
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import math

In [3]:
%matplotlib inline

# initialization of flux values so i can debug:


In [7]:




####Defining the constants

#define coordinates	
ra= 359.45700000000016
dec= -32.592000000000013
pos = SkyCoord(ra=ra, dec=dec, unit= 'deg')
print(pos)

# Lookup and define a service for ALLWISE Atlas images
# the website gave me the URL
allwise_service = vo.dal.SIAService("https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?")

#search the service for images covering within 1 arcsecond of the star. make this bigger if needed
im_table = allwise_service.search(pos=pos, size= 1*u.arcsec)
#im_table
im_table.to_table().colnames
for i in range(len(im_table)):
    print(im_table[i])


   #grab the x-ray sources for this galaxy
#import huge csv and grab the name and ra and dec needed for each source.
targetgals = pd.read_csv('../Data/inWISE.csv') # this should not be the one for all 120 and should rather be for the 74 of them.
print(targetgals[0:20])
huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')
columns = ['RA','Dec', 'Gname_Modified','Gname_Homogenized']
g_huge = huge[columns]
g_huge[0:100]

#locate through merging
df1 = targetgals
df2 = g_huge

merged_data = pd.merge(df1, df2, left_on='source_id', right_on = 'Gname_Homogenized', how='inner')
columns = ['RA','Dec','Gname_Homogenized']
Xray_sources = merged_data[columns]
Xray_sources

# just want NGC 7793s sources
sources_7793 = Xray_sources.loc[Xray_sources['Gname_Homogenized'] == 'NGC 7793']
#sources_7793
#define the X-ray sources of NGC 7793
ra = sources_7793['RA'].values
dec = sources_7793['Dec'].values
#print(ra)

# defining a function to calculate the distances between two sources.
def dist(p1, p2):
    return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )

#defining a function that creates a circular mask around each source so that if something overlaps with it, that overlapping part is not included in the aperture photometry
def create_circular_mask(h,w,center,radius):
    Y, X = np.ogrid[:h, :w] # creating an open (more memory efficient) coordinate grid of the image
    dist_from_center = np.sqrt((X-center[0])**2+ (Y-center[1])**2)
    mask = dist_from_center <= radius # so that everything inside the radius receives a mask
    return mask

# define a mapping of the bands into labels to make it easier
band_labels = {'w1': 'W1', 'w2': 'W2', 'w3': 'W3', 'w4': 'W4'}
flux_zmfd = {'w1': 309.54 ,'w2': 171.787,'w3': 31.674,'w4': 8.363} # check if these worked by looking at the band 4 code above
instr_zpmag = {'w1': 20.73,'w2': 19.56,'w3': 17.6 ,'w4':12.98 }
wavelengths = {'w1': 3.4 ,'w2': 4.6,'w3': 12,'w4': 22}

#define function to get flux density per unit frequency (energy units)
def flux_dens(net_flx, net_flx_err, wavelength):
    flux_density = (net_flx * 1e-23) * (3e10 / (wavelength*1e-4)**2)
    flux_density_unc = (net_flx_err * 1e-23) * (3e10 / (wavelength*1e-4)**2)
    return flux_density, flux_density_unc


##running the for loop over every image and doing aperture photometry on each one
#currently outputs as w4,w1,w2,w3 when querying the images. so index is 0.1.2.3 i want the index to be 0.3.2.1
rows = []
for i in [0, 3, 2, 1]:  # Reverse order index
    band_id = im_table[i]["sia_bp_id"].lower()  # Get band ID in lowercase

    if band_id in band_labels:
        print(f'Band {band_labels[band_id]}: ')
        data_url = im_table[i].getdataurl()
        #Download the image and open it in Astropy
        fname = download_file(data_url, cache=True)
        image1= fits.open(fname)
        image_data= image1[0].data
        #print(data)
        
        wcs = WCS(image1[0].header)
        #cuting out the image of the galaxy apart from the rest of the background.
        cutout = Cutout2D(image_data, pos, (437,437), wcs=wcs)
        wcs = cutout.wcs
        cutout_data = cutout.data
        #print(cutout_data)
        positions = wcs.world_to_pixel_values(ra, dec)
        positions = np.array(list(zip(positions[0], positions[1])))

        #define the distance threshold for the KDTree grouping (in pixels)
        distance_threshold = 5

        #build the KDTree for efficient grouping
        tree = KDTree(positions)

        #query the KDTree to find points within the defined radius of dist threshold and group them together
        groups = tree.query_ball_tree(tree, r=distance_threshold)
       # print(groups)
        # consolidating the groups. 'unique_groups' and 'seen': These are used to ensure that each group is processed only once.
        unique_groups = []
        seen = set()
        for group in groups:
            group = tuple(sorted(group))
            if group not in seen:
                seen.add(group)
                unique_groups.append(group)
       # print(unique_groups)
        # for each unique group, the average postion of the detections is calulated so that only one source detection is used for aperture photometry instead of a bunch of the same sources being used.
        #represents the consolidated postion of potentially multiple detections of one source.
        grouped_positions = [positions[list(group)].mean(axis=0) for group in unique_groups]
        #print(grouped_positions)

        #define the Region(s) Of Interest (center x, center y, radius)
        ROI = [ ((x, y) , 9, 16, 23) for x,y in grouped_positions ] # (x, y, radius around target, inner r, outer r)   36.3636, 50.90909) may need to mke the radius bigger when goruping? 
        

        # initialize valid rows plotting for the current image iteration
        valid_rows = []
        
        #now inputting the aperture photometry part
        # check for overlap and perform aperture photometry
        for i, ((x, y), r, annulus_inner, annulus_outer) in  enumerate(ROI):
            overlap = False# initialize overlap flag (A boolean flag overlap is set to False for each source to track if it overlaps with any other source.)
            non_overlapping_mask = create_circular_mask(cutout_data.shape[0], cutout_data.shape[1], (x,y), r)

            for j, ((x2, y2), r2, annulus_inner2, annulus_outer2) in  enumerate(ROI): # apparently you can run a for loop over 2 objects by putting the second one inside the first. it iterates over every source again to then see if it overlaps at all with another source.
                if i != j: # ensures that a source is not compared to itself! wow
                    distance = dist((x, y) , (x2, y2)) 
                    if distance < r + r2:  # if the distance is less than the size of the two radii added together, then they are overlapping.
                        overlap_percent = (r + r2 - distance)/(r+r2)  # the amount they are overlapping divided by the total size of radii distance
                        if overlap_percent > .5:
                            overlap = True # this way, if they overlap by more than 50% then they will not be usable because less than 50% of the flux extractable area can be seen.
                            overlap_mask = create_circular_mask(cutout_data.shape[0], cutout_data.shape[1], (x2,y2), r2)
                            non_overlapping_mask = np.logical_and(non_overlapping_mask, np.logical_not(overlap_mask)) # so that the overlapping part is read as false and thus excluded from calculation of flux.
                            # "logical and and not" are used to exclude certain regions such as the overlapping part! the logical not makes it so that 
                            # values that were once true in the overlap mask are not false, and the logical_and is there so that only the true values are accepted. effectively rejecting the part the overlapping mask covers.

                            #Handle overlaps that are acceptable (less than the threshold, but still overlapping by a small percent)
                            overlap_aperture = CircularAperture((x2, y2), r2)
                            overlap_photo_table = aperture_photometry(cutout_data, overlap_aperture)
                            overlap_counts = overlap_photo_table['aperture_sum'][0] 
                            overlap_error = np.sqrt(overlap_counts)
                            
                        else: 
                            overlap_mask = create_circular_mask(cutout_data.shape[0], cutout_data.shape[1], (x2,y2), r2)
                            non_overlapping_mask = np.logical_and(non_overlapping_mask, np.logical_not(overlap_mask))
                            overlap_aperture = CircularAperture((x2, y2), r2)
                            overlap_photo_table = aperture_photometry(cutout_data, overlap_aperture)
                            overlap_counts = overlap_photo_table['aperture_sum'][0] 
                            overlap_error = np.sqrt(overlap_counts)
                            
            if overlap:
                # For the Target objects in the little aperture circle define their target apertures
                target_aperture = CircularAperture((x,y),r,)
            
                #perform aperture photometry on target
                target_photo_table = aperture_photometry(cutout_data, target_aperture) 
                target_counts = target_photo_table['aperture_sum'][0]
                target_counts -= overlap_counts

                # so that i dont take the log or sqrt of a negative number or zero and get an error
                if target_counts > 0: # 
                    target_error= np.sqrt(target_counts)
                    #propagated error of overlap error
                    target_overlap_counts_err = np.sqrt(target_error**2 + overlap_error**2)
                    #print(target_counts)
                    if band_id in flux_zmfd and instr_zpmag: 
                         #print(f'Band {flux_zmfd[band_id]}: ')
                         flx_conv_fact = flux_zmfd[band_id]
                         M0instr = instr_zpmag[band_id]
                         Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                         target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux
                         
                         #propagation of uncertainty of flux conversion
                         Mcal_error = (2.5*target_overlap_counts_err) / (target_counts * np.log(10))
                         target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)

                else:
                         # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        rows.append({ 'band_id': {band_labels[band_id]},'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 
                        'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx, 'Flux Uncertainty': [], 'Flag':'Negative Target Counts',
                        'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                            'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                            'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                            'Offset in Arcseconds': [],  'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : []  })

                    
                #calculate area of target aperutue
                target_area = target_aperture.area




                # For the Background Annuli of outside of the Target
                #define the background annulus for the target
                annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                #perform aperture photometry on annuli
                annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                annulus_counts = annulus_photo_table['aperture_sum'][0]
                
            
                if annulus_counts > 0:
                    overlapannulus_error = np.sqrt(annulus_counts) # the error of the annulus for sources that overlap
                     # to avoid taking the log of zero or negative value
                    if band_id in flux_zmfd and instr_zpmag: 
                         #print(f'Band {flux_zmfd[band_id]}: ')
                         flx_conv_fact = flux_zmfd[band_id]
                         M0instr = instr_zpmag[band_id]
                         Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                         annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                         #propagation of uncertainty of flux conversion
                         Mcal_error_ann = (2.5 * overlapannulus_error) / (annulus_counts * np.log(10))
                         annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)


                else:
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        
                #calculate area of annulus
                annulus_area = annulus_aperture.area

                # do the calculations for including a Background aperture
            
                #Calculating the net flux:
                #calculate the mean background per pixel
                bg_perpixel = annulus_flux/annulus_area
                bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                #calculate the total background in the target aperture
                tot_bg = bg_perpixel * target_area
                tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                #Subtract background from the target flux
                net_flx = target_flux - tot_bg
                net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!
            
                #flag the sources that overlap
                rows.append({ 'band_id': {band_labels[band_id]},'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 
                        'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': net_flx_err, 'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux',
                        'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                            'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                            'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 
                            'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                # Append valid results to valid_rows
                valid_rows.append({
                    'band_id': {band_labels[band_id]},
                    'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                    'Annulus_Inner_Radius': annulus_inner,
                    'Annulus_Outer_Radius': annulus_outer,
                    'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux', 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 
                    'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [],  'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                
            else: #perform all the aperture photometry stuff
                # For the Target objects in the little aperture circle define their target apertures
                target_aperture = CircularAperture((x,y),r,)
            
                #perform aperture photometry on target
                target_photo_table = aperture_photometry(cutout_data, target_aperture) 
                target_counts = target_photo_table['aperture_sum'][0]

                if target_counts > 0: # 
                    # avoid taking the log of zero or negative value
                    target_error = np.sqrt(target_counts)
                    if band_id in flux_zmfd and instr_zpmag: 
                         #print(f'Band {flux_zmfd[band_id]}: ')
                         flx_conv_fact = flux_zmfd[band_id]
                         M0instr = instr_zpmag[band_id]
                         Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                         target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                         #propagation of uncertainty of flux conversion
                         Mcal_error = (2.5*target_error) / (target_counts * np.log(10))
                         target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)

                else:
                         # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        rows.append({ 'band_id': {band_labels[band_id]},'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 
                        'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': [],'Flag':'Negative Target Counts',
                        'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                            'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                            'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                            'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })

                    
                #calculate area of target aperutue
                target_area = target_aperture.area




                # For the Background Annuli of outside of the Target
                #define the background annulus for the target
                annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                #perform aperture photometry on annuli
                annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                annulus_counts = annulus_photo_table['aperture_sum'][0]
                # the error of the annulus for sources that overlap
                
                if annulus_counts > 0:
                    annulus_error = np.sqrt(annulus_counts)
                     # avoid taking the log of zero or negative value
                    if band_id in flux_zmfd and instr_zpmag: 
                         #print(f'Band {flux_zmfd[band_id]}: ')
                         flx_conv_fact = flux_zmfd[band_id]
                         M0instr = instr_zpmag[band_id]
                         Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                         annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                         #propagation of uncertainty of flux conversion
                         Mcal_error_ann = (2.5 * annulus_error) / (annulus_counts * np.log(10))
                         annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)
                else:
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        
               #calculate area of annulus
                annulus_area = annulus_aperture.area

                # do the calculations for including a Background aperture
            
                #Calculating the net flux:
                #calculate the mean background per pixel
                bg_perpixel = annulus_flux/annulus_area
                bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                #calculate the total background in the target aperture
                tot_bg = bg_perpixel * target_area
                tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                #Subtract background from the target flux
                net_flx = target_flux - tot_bg
                net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!

                #   Append the result as a dictionary to the list named 'rows'
                rows.append({ 'band_id': {band_labels[band_id]},'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 
                        'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,  'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux',
                        'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                            'Annulus Counts': annulus_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                            'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                              'Offset in Arcseconds': [],   'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id],'Flux Density': [], 'Flux Density Uncertainty' : [] })  #will prolly have to change the name of the Flux here!!!
            #"Low Flux" means that they either have zero flux or negative flux and so they are excluded from the plotting
               
                # Append valid results to valid_rows
                valid_rows.append({
                    'band_id': {band_labels[band_id]},
                    'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                    'Annulus_Inner_Radius': annulus_inner,
                    'Annulus_Outer_Radius': annulus_outer,
                    'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,
                    'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux', 
                    'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 'Exists?': 'No', 'Point Source Position' : [], 'Offset in Arcseconds': [],
                     'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })


        #Source detection code
        mean, median, std = sigma_clipped_stats(cutout_data, sigma=3.0)  
       #print((mean, median, std))

        # subtract the background and find FWHM of sources at a certain threshold
        #started at fwhm= 3 and threshold = 5
        daofind = DAOStarFinder(fwhm=8, threshold=1*std) # find the stars in the image that have FWHMs of around 3 pixels and have peaks approximately 5-sigma above the background. 
        sources = daofind(cutout_data - median)  
        #print(type(sources))
        # will likely run into iissues in the code below
        for col in sources.colnames:  
            if col not in ('id', 'npix'):
                sources[col].info.format = '%.2f'  # for consistent table output
       # sources.pprint(max_width=3000)  

        #likely the flux labeled in this is not converted!
        
        # plot the image with marked locations of all the sources it detected.
        detected_positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
        apertures = CircularAperture(detected_positions, r=2)


        # Plotting for current image
        # Filter valid rows
        valid_rows_filtered = [row for row in valid_rows if row['Flag'] == 'Valid']
        
        # Was there a point source there?
        pixelsinarc = 0.0003819444391411 * 3600 ## 0.0003819444391411 is the number found in the header of the image for the scale of pixels in degrees for the fits image.
        detectedpos_all = []
        #rows = [row for row in rows if row['Flag'] == 'Valid']
        for row in valid_rows_filtered:
            apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
            for detected_position in detected_positions:
                detected_x, detected_y = detected_position
                distance = dist((row['X'], row['Y']), (detected_x, detected_y) ) #distance from the center of the xray source to the center of the detected source
                if distance <= row['Radius']: 
                    row['Exists?'] = 'Point Source Detected'
                    row['Point Source Position'].append((detected_x, detected_y))
                    dist_in_arc = distance * pixelsinarc
                    row['Offset in Arcseconds'].append((dist_in_arc))
                    detectedpos_all.append(detected_position) 
                    #print(f"Number of detected positions within forced apertures: {len(detectedpos_all)}")


        Yes = []
        YesRadius= 5
        for row in valid_rows_filtered:
                    apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                    for detected_position in detected_positions:
                        detected_x, detected_y = detected_position
                        distance = dist((row['X'], row['Y']), (detected_x, detected_y) )

                        if distance <= YesRadius:
                            row['Exists?'] = 'Yes!!'
                            Yes.append((row['X'], row['Y']))
        #doing flux density
        for row in valid_rows_filtered:
             net_flx = row['Net Flux (Jy)']
             net_flx_err = row['Flux Uncertainty']
             wavelength = row['Wavelength']
             #print(net_flx)
             flux_density, flux_density_unc = flux_dens(net_flx, net_flx_err, wavelength)
             row['Flux Density'].append(flux_density)
             row['Flux Density Uncertainty'].append(flux_density_unc)
                 
        
        # Update rows with valid_rows_filtered information
        for valid_row in valid_rows_filtered:
            for row in rows:
                if row['band_id'] == valid_row['band_id'] and row['Region'] == valid_row['Region']:
                    row.update(valid_row)

        #print('valid sources', valid_rows_filtered)

        #print( len(detectedpos_all))
        if len(detectedpos_all) > 0:
            apertures_detected = CircularAperture(detectedpos_all, r=2)
        if len(Yes) > 0:
            apertures_Yes = CircularAperture(Yes, r=YesRadius)
        
       # xc = 244.422687	 #19.014239	
       # yc=  191.596758# 310.340772
    
'''
         # Plotting for current image
        fig, ax = plt.subplots(subplot_kw={'projection': wcs})
        norm = ImageNormalize(cutout.data, stretch=SqrtStretch())
        for row in valid_rows_filtered:
            target_aperture = CircularAperture((row['X'], row['Y']), row['Radius'])
            annulus_aperture = CircularAnnulus((row['X'], row['Y']), row['Annulus_Inner_Radius'], row['Annulus_Outer_Radius'])
            target_aperture.plot(color='red', lw=1.5, alpha=.5, ax=ax)
            annulus_aperture.plot(color='blue', lw=1.5, alpha=.5, ax=ax)
            
            apertures.plot(color='#98ff98', lw=.5, alpha=0.5) 
            apertures_detected.plot(color='#FF4500', lw=.5, alpha=0.5)  #  #FF69B4 for hot pink
            apertures_Yes.plot(color='green', lw=.5, alpha=0.5) 

            # curious ones
           # curious = CircularAperture((xc,yc),5)
           # curious.plot(color='red', lw=.5, alpha=0.5)
        ax.imshow(cutout.data, cmap='gray', norm=norm, interpolation='nearest')
        ax.set_xlabel('Right Ascension')
        ax.set_ylabel('Declination')
        plt.title(f'Band {band_labels[band_id]}')
        plt.show()
        yesses =[row for row in valid_rows_filtered if row['Exists?'] == 'Yes!!']
       '''

       

        
        



display_data = pd.DataFrame(rows)
display_data
#pd.set_option("display.max_rows", None)
#pd.set_option("display.max_columns", None)
print(len(display_data.loc[display_data['Flag']== 'Valid']))
print(len(display_data))
#display(display_data.loc[display_data['Flag']== 'Valid'])
#display(display_data)
print(len(display_data.loc[display_data['Flag']== 'Valid']))



<SkyCoord (ICRS): (ra, dec) in deg
    (359.457, -32.592)>
('W4 Coadd 0000m334_ac51', 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/00/0000/0000m334_ac51/0000m334_ac51-w4-int-3.fits', '{"aws": {"bucket_name": "nasa-irsa-wise", "key":"wise/allwise/images/p3am_cdd/00/0000/0000m334_ac51/0000m334_ac51-w4-int-3.fits", "region": "us-west-2"}}', 2, 'image/fits', 0.0, -33.317778, array([4095, 4095], dtype=int32), array([2048., 2048.]), array([  0.      , -33.317778]), 'SIN', array([-0.00038194,  0.00038194]), array([-0.00038194, -0.        , -0.        ,  0.00038194]), 'W4', 2.209e-05, 2.336e-05, 1.984e-05, 'm', 13.0, 0.012, 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/00/0000/0000m334_ac51/0000m334_ac51-w4-unc-3.fits.gz', 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/00/0000/0000m334_ac51/0000m334_ac51-w4-cov-3.fits.gz', '0000m334_ac51')
('W1 Coadd 0000m334_ac51', 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/00/0000/0000m334_ac

/var/folders/nd/77pj43rx7wbcbp0wx2qjq2140000gp/T/ipykernel_38896/507410234.py:25: DtypeWarning: Columns (533,534) have mixed types. Specify dtype option on import or set low_memory=False.
  huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')


Band W4: 
Band W3: 
Band W2: 
Band W1: 
109
780
109


# Debugging:

In [14]:
####Defining the constants
# defining a function to calculate the distances between two sources.
def dist(p1, p2):
   return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )


#defining a function that creates a circular mask around each source so that if something overlaps with it, that overlapping part is not included in the aperture photometry
def create_circular_mask(h,w,center,radius):
   Y, X = np.ogrid[:h, :w] # creating an open (more memory efficient) coordinate grid of the image
   dist_from_center = np.sqrt((X-center[0])**2+ (Y-center[1])**2)
   mask = dist_from_center <= radius # so that everything inside the radius receives a mask
   return mask


# define a mapping of the bands into labels to make it easier
band_labels = {'w1': 'W1', 'w2': 'W2', 'w3': 'W3', 'w4': 'W4'}
flux_zmfd = {'w1': 309.54 ,'w2': 171.787,'w3': 31.674,'w4': 8.363} # check if these worked by looking at the band 4 code above
instr_zpmag = {'w1': 20.73,'w2': 19.56,'w3': 17.6 ,'w4':12.98 }
wavelengths = {'w1': 3.4 ,'w2': 4.6,'w3': 12,'w4': 22}


#define function to get flux density per unit frequency (energy units)
def flux_dens(net_flx, net_flx_err, wavelength):
   flux_density = (net_flx * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   flux_density_unc = (net_flx_err * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   return flux_density, flux_density_unc


'''
make a conditional code where:
choose one obs id and eerything associated with it bwith the longest exposuretime exptime?
tell me all of the obsids associated with this galaxy_name
    unique list of each galaxy Name
    what are the obs ids associated with it
    if more than one which one is the longest exposure
    aggregate all of the associated sources with it and none from any of the others


'''


#import huge csv and grab the name and ra and dec needed for each galaxy.
targetgals = pd.read_csv('../Data/inWISE.csv') # this should not be the one for all 120 and should rather be for the 74 of them.
#print(targetgals[0:20])
huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')
columns = ['RA','Dec','Gname_Modified','Gname_Homogenized', 'ObsID', 'EXPOSURE']
g_huge = huge[columns]
#display(g_huge.head())


#group the x-ray sources for this galaxy. locate through merging
df1 = targetgals
df2 = g_huge




merged_data = pd.merge(df1, df2, left_on='source_id', right_on = 'Gname_Homogenized', how='inner')
columns1 = ['RA','Dec','Gname_Homogenized', 'ObsID', 'EXPOSURE']
Xray_sources = merged_data[columns1]


#group by galaxy name and the longest exposure time.
longest_exposure_obs = Xray_sources.loc[Xray_sources.groupby('Gname_Homogenized')['EXPOSURE'].idxmax()]


# aggregate all the sources associated with the obsid with the longest exposure time
aggregated_sources = Xray_sources[Xray_sources['ObsID'].isin(longest_exposure_obs['ObsID'])]




#align them based on source Id since they all got jumbled.
#aligned_targetgals = targetgals[targetgals.isin(aggregated_sources['Gname_Homogenized'])]


#reset index and group them by name
#aligned_targetgals = aligned_targetgals.reset_index(drop=True)
#print(aligned_targetgals)
#aligned_aggregatedsources = aggregated_sources.groupby('Gname_Homogenized')






'''
print("Unique list of each galaxy name:")
print(targetgals['source_id'])


print("\nObservations with the longest exposure time for each galaxy:")
print(longest_exposure_obs)


print("\nAggregated sources:")
print(aggregated_sources)
#'''












# Ensure the uniqueness for NGC 5128
#ngc_5128_sources = Xray_sources[Xray_sources['Gname_Homogenized'] == 'NGC 5128']
#print(f"Number of unique X-ray sources for NGC 5128: {len(ngc_5128_sources)}")
#print(ngc_5128_sources)


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#display(g_huge)
#display(Xray_sources)








#create a list of all the names needed
galaxy_names = targetgals['source_id'].unique()




galaxy_sources = {}
grouped_sources = aggregated_sources.groupby('Gname_Homogenized')
'''print("\nGrouped sources:")
for group_name, group in grouped_sources:
   print(f"\nGroup: {group_name}")
   print(group)'''
#get all of the ra and dec sources for the galaxy in question
for group_name, group in grouped_sources:
   galaxy_sources[group_name] = {'ra' : group['RA'].values, 'dec' : group['Dec'].values, 'ObsID': group['ObsID'].values}
  






#### initialize everything

#overlap_percent = []
#distance = []
























rows = []
#create subsets for testing:
grouped_sources_subset = grouped_sources.head(2)
targetgals_subset = targetgals.head(2)


#targetgals_subset = targetgals.iloc[17:18]


# Lookup and define a service for ALLWISE Atlas images
allwise_service = vo.dal.SIAService("https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?")
#loop through the galaxies
print("\nAligned target galaxies and grouped sources:")


for galaxy in targetgals_subset.itertuples():
   galaxy_name = galaxy.source_id
   print(galaxy_name)
   #group = aligned_aggregatedsources.get_group(galaxy_name)
  
   # Print galaxy information
   #print(f"\nGalaxy: {galaxy_name}")
   #print(group)


   #define coordinates
   ra1 = galaxy.ra_x
  #print (ra1)
   dec1 = galaxy.dec_x
   pos = SkyCoord(ra=ra1, dec=dec1, unit= 'deg')
   #search the service for images covering within 1 arcsecond of the star. make this bigger if needed
   im_table = allwise_service.search(pos=pos, size= 1*u.arcsec)
   #im_table
   im_table.to_table().colnames
   print(im_table)
   # get the Ra and dec values necessary for the kdtree and rest of the code
  
   if galaxy_name in galaxy_sources:
       ra = galaxy_sources[galaxy_name]['ra']
       dec = galaxy_sources[galaxy_name]['dec']
       obsid = galaxy_sources[galaxy_name]['ObsID']
       print(f"Galaxy: {galaxy_name}")
       print("RA values:", ra)
       print("Number of RA values:", len(ra))
   else:
       print(f"No sources found for galaxy: {galaxy_name}")
          
   ##running the for loop over every image and doing aperture photometry on each one
   #currently outputs as w4,w1,w2,w3 when querying the images. so index is 0.1.2.3 i want the index to be 0.3.2.1


   for i in [0, 3, 2, 1]:  # Reverse order index
       band_id = im_table[i]["sia_bp_id"].lower()  # Get band ID in lowercase


       if band_id in band_labels:
           print(f'Band {band_labels[band_id]}: ')
           data_url = im_table[i].getdataurl()
           #Download the image and open it in Astropy
           fname = download_file(data_url, cache=True)
           image1= fits.open(fname)
           image_data= image1[0].data
           #print(data)
           print(data_url)
           wcs = WCS(image1[0].header)
           #cuting out the image of the galaxy apart from the rest of the background.
           cutout = Cutout2D(image_data, pos, (437,437), wcs=wcs)
           wcs = cutout.wcs
           cutout_data = cutout.data
           #print(cutout_data)
           positions = wcs.world_to_pixel_values(ra, dec)
           positions = np.array(list(zip(positions[0], positions[1])))


           #define the distance threshold for the KDTree grouping (in pixels)
           distance_threshold = 6


           #build the KDTree for efficient grouping
           tree = KDTree(positions)


           #query the KDTree to find points within the defined radius of dist threshold and group them together
           groups = tree.query_ball_tree(tree, r=distance_threshold)
       # print(groups)
           # consolidating the groups. 'unique_groups' and 'seen': These are used to ensure that each group is processed only once.
           unique_groups = []
           seen = set()
           for group in groups:
               group = tuple(sorted(group))
               if group not in seen:
                   seen.add(group)
                   unique_groups.append(group)
       # print(unique_groups)
           # for each unique group, the average postion of the detections is calulated so that only one source detection is used for aperture photometry instead of a bunch of the same sources being used.
           #represents the consolidated postion of potentially multiple detections of one source.
           grouped_positions = [positions[list(group)].mean(axis=0) for group in unique_groups]
           #print(grouped_positions)


           #define the Region(s) Of Interest (center x, center y, radius)
           ROI = [ ((x, y) , 9, 16, 23) for x,y in grouped_positions ] # (x, y, radius around target, inner r, outer r)   36.3636, 50.90909) may need to mke the radius bigger when goruping?
          


           # initialize valid rows plotting for the current image iteration
           valid_rows = []
          
           #now inputting the aperture photometry part
           # check for overlap and perform aperture photometry
           for i, ((x, y), r, annulus_inner, annulus_outer) in  enumerate(ROI):
               overlap = False # initialize overlap flag (A boolean flag overlap is set to False for each source to track if it overlaps with any other source. becomes true if they do overlap 
               non_overlapping_mask = create_circular_mask(cutout_data.shape[0], cutout_data.shape[1], (x,y), r)

               acc_overlap = False # initialize the acceptable overlap flag.  false if there is no overlap, true if there is overlap and it is acceptable 


               for j, ((x2, y2), r2, annulus_inner2, annulus_outer2) in  enumerate(ROI): # apparently you can run a for loop over 2 objects by putting the second one inside the first. it iterates over every source again to then see if it overlaps at all with another source.
                   if i != j: # ensures that a source is not compared to itself! wow
                       distance = dist((x, y) , (x2, y2))
                       print('dsitance', distance)
                       if distance < r + r2:  # if the distance is less than the size of the two radii added together, then they are overlapping.
                           overlap_percent = (r + r2 - distance)/(r+r2)  # the amount they are overlapping divided by the total size of radii distance
                           print('overlap perc', overlap_percent)
                           if overlap_percent > .5:
                               overlap = True # this way, if they overlap by more than 50% then they will not be usable because less than 50% of the flux extractable area can be seen.
                               print('overlap is: ', overlap)
                               overlap_mask = np.nan
                               non_overlapping_mask = np.nan
                               overlap_aperture = np.nan
                               overlap_photo_table = np.nan
                               overlap_counts = np.nan
                               overlap_error = np.nan
                              
                           else:
                               if overlap_percent < .5:
                                    acc_overlap = True
                                    print('acceptable or no overlap: ', acc_overlap)
                                    overlap_mask = create_circular_mask(cutout_data.shape[0], cutout_data.shape[1], (x2,y2), r2)
                                    non_overlapping_mask = np.logical_and(non_overlapping_mask, np.logical_not(overlap_mask)) # so that the overlapping part is read as false and thus excluded from calculation of flux.
                                        # "logical and and not" are used to exclude certain regions such as the overlapping part! the logical not makes it so that
                                        # values that were once true in the overlap mask are not false, and the logical_and is there so that only the true values are accepted. effectively rejecting the part the overlapping mask covers.

                                     #Handle overlaps that are acceptable (less than the threshold, but still overlapping by a small percent)
                                    overlap_aperture = CircularAperture((x2, y2), r2)
                                    overlap_photo_table = aperture_photometry(cutout_data, overlap_aperture)
                                    overlap_counts = overlap_photo_table['aperture_sum'][0]
                                    overlap_error = np.sqrt(overlap_counts)


                       else:
                           print('Ran else')
               if acc_overlap:
                   # For the Target objects in the little aperture circle define their target apertures
                   target_aperture = CircularAperture((x,y),r,)
              
                   #perform aperture photometry on target
                   target_photo_table = aperture_photometry(cutout_data, target_aperture)
                   target_counts = target_photo_table['aperture_sum'][0]
                   target_counts -= overlap_counts


                   # so that i dont take the log or sqrt of a negative number or zero and get an error
                   if target_counts > 0: #
                       target_error= np.sqrt(target_counts)
                       #propagated error of overlap error
                       target_overlap_counts_err = np.sqrt(target_error**2 + overlap_error**2)
                       print(target_counts)
                       if band_id in flux_zmfd and instr_zpmag:
                           #print(f'Band {flux_zmfd[band_id]}: ')
                           flx_conv_fact = flux_zmfd[band_id]
                           M0instr = instr_zpmag[band_id]
                           Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                           target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux
                          
                           #propagation of uncertainty of flux conversion
                           Mcal_error = (2.5*target_overlap_counts_err) / (target_counts * np.log(10))
                           target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)


                   else:
                           # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                           rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx, 'Flux Uncertainty': [], 'Flag':'Negative Target Counts',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                               'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                               'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [],  'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : []  })


                      
                   #calculate area of target aperutue
                   target_area = target_aperture.area








                   # For the Background Annuli of outside of the Target
                   #define the background annulus for the target
                   annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)


                   #perform aperture photometry on annuli
                   annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                   annulus_counts = annulus_photo_table['aperture_sum'][0]
                  
              
                   if annulus_counts > 0:
                       overlapannulus_error = np.sqrt(annulus_counts) # the error of the annulus for sources that overlap
                       # to avoid taking the log of zero or negative value
                       if band_id in flux_zmfd and instr_zpmag:
                           #print(f'Band {flux_zmfd[band_id]}: ')
                           flx_conv_fact = flux_zmfd[band_id]
                           M0instr = instr_zpmag[band_id]
                           Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                           annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux


                           #propagation of uncertainty of flux conversion
                           Mcal_error_ann = (2.5 * overlapannulus_error) / (annulus_counts * np.log(10))
                           annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)




                   else:
                           annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                          
                   #calculate area of annulus
                   annulus_area = annulus_aperture.area


                   # do the calculations for including a Background aperture
              
                   #Calculating the net flux:
                   #calculate the mean background per pixel
                   bg_perpixel = annulus_flux/annulus_area
                   bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!


                   #calculate the total background in the target aperture
                   tot_bg = bg_perpixel * target_area
                   tot_bg_err = bg_perpixel_err * target_area ##propagation of error!


                   #Subtract background from the target flux
                   net_flx = target_flux - tot_bg
                   net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!
              
                   #flag the sources that overlap
                   rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': net_flx_err, 'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                               'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                               'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                   # Append valid results to valid_rows
                   valid_rows.append({
                       'band_id': {band_labels[band_id]},  'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0],
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux', 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                       'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [],  'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                  
               else: #perform all the normal aperture photometry stuff for those that do not overlap in any way.
                   # For the Target objects in the little aperture circle define their target apertures
                   target_aperture = CircularAperture((x,y),r,)
              
                   #perform aperture photometry on target
                   target_photo_table = aperture_photometry(cutout_data, target_aperture)
                   target_counts = target_photo_table['aperture_sum'][0]


                   if target_counts > 0: #
                       # avoid taking the log of zero or negative value
                       target_error = np.sqrt(target_counts)
                       if band_id in flux_zmfd and instr_zpmag:
                           #print(f'Band {flux_zmfd[band_id]}: ')
                           flx_conv_fact = flux_zmfd[band_id]
                           M0instr = instr_zpmag[band_id]
                           Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                           target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux


                           #propagation of uncertainty of flux conversion
                           Mcal_error = (2.5*target_error) / (target_counts * np.log(10))
                           target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)


                   else:
                           # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                           rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': [],'Flag':'Negative Target Counts',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                               'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                               'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })


                      
                   #calculate area of target aperutue
                   target_area = target_aperture.area








                   # For the Background Annuli of outside of the Target
                   #define the background annulus for the target
                   annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)


                   #perform aperture photometry on annuli
                   annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                   annulus_counts = annulus_photo_table['aperture_sum'][0]
                   # the error of the annulus for sources that overlap
                  
                   if annulus_counts > 0:
                       annulus_error = np.sqrt(annulus_counts)
                       # avoid taking the log of zero or negative value
                       if band_id in flux_zmfd and instr_zpmag:
                           #print(f'Band {flux_zmfd[band_id]}: ')
                           flx_conv_fact = flux_zmfd[band_id]
                           M0instr = instr_zpmag[band_id]
                           Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                           annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux


                           #propagation of uncertainty of flux conversion
                           Mcal_error_ann = (2.5 * annulus_error) / (annulus_counts * np.log(10))
                           annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)
                   else:
                           annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                          
               #calculate area of annulus
                   annulus_area = annulus_aperture.area


                   # do the calculations for including a Background aperture
              
                   #Calculating the net flux:
                   #calculate the mean background per pixel
                   bg_perpixel = annulus_flux/annulus_area
                   bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!


                   #calculate the total background in the target aperture
                   tot_bg = bg_perpixel * target_area
                   tot_bg_err = bg_perpixel_err * target_area ##propagation of error!


                   #Subtract background from the target flux
                   net_flx = target_flux - tot_bg
                   net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!


                   #   Append the result as a dictionary to the list named 'rows'
                   rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,  'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux,
                               'Annulus Counts': annulus_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Mask Shape': non_overlapping_mask.shape,
                               'Mask Type': non_overlapping_mask.dtype, 'Non-zero mask elements': np.count_nonzero(non_overlapping_mask), 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [],   'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id],'Flux Density': [], 'Flux Density Uncertainty' : [] })  #will prolly have to change the name of the Flux here!!!
               #"Low Flux" means that they either have zero flux or negative flux and so they are excluded from the plotting
              
                   # Append valid results to valid_rows
                   valid_rows.append({
                       'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0],
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,
                       'Flag':'Valid' if not math.isnan(target_counts) and target_counts > 0 and net_flx > 0 else 'Low Flux',
                       'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 'Exists?': 'No', 'Point Source Position' : [], 'Offset in Arcseconds': [],
                       'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })




           #Source detection code
           mean, median, std = sigma_clipped_stats(cutout_data, sigma=3.0) 
       #print((mean, median, std))


           # subtract the background and find FWHM of sources at a certain threshold
           #started at fwhm= 3 and threshold = 5
           daofind = DAOStarFinder(fwhm=8, threshold=1*std) # find the stars in the image that have FWHMs of around 3 pixels and have peaks approximately 5-sigma above the background.
           sources = daofind(cutout_data - median) 
           #print(type(sources))
           # will likely run into iissues in the code below
           for col in sources.colnames: 
               if col not in ('id', 'npix'):
                   sources[col].info.format = '%.2f'  # for consistent table output
       # sources.pprint(max_width=3000) 


           #likely the flux labeled in this is not converted!
          
           # plot the image with marked locations of all the sources it detected.
           detected_positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
           apertures = CircularAperture(detected_positions, r=2)




           # Plotting for current image
           # Filter valid rows
           valid_rows_filtered = [row for row in valid_rows if row['Flag'] == 'Valid']
          
           # Was there a point source there?
           pixelsinarc = 0.0003819444391411 * 3600 ## 0.0003819444391411 is the number found in the header of the image for the scale of pixels in degrees for the fits image.
           detectedpos_all = []
           #rows = [row for row in rows if row['Flag'] == 'Valid']
           for row in valid_rows_filtered:
               apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
               for detected_position in detected_positions:
                   detected_x, detected_y = detected_position
                   distance = dist((row['X'], row['Y']), (detected_x, detected_y) ) #distance from the center of the xray source to the center of the detected source
                   if distance <= row['Radius']:
                       row['Exists?'] = 'Point Source Detected'
                       row['Point Source Position'].append((detected_x, detected_y))
                       dist_in_arc = distance * pixelsinarc
                       row['Offset in Arcseconds'].append((dist_in_arc))
                       detectedpos_all.append(detected_position)
                       #print(f"Number of detected positions within forced apertures: {len(detectedpos_all)}")




           Yes = []
           YesRadius= 5
           for row in valid_rows_filtered:
                       apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                       for detected_position in detected_positions:
                           detected_x, detected_y = detected_position
                           distance = dist((row['X'], row['Y']), (detected_x, detected_y) )


                           if distance <= YesRadius:
                               row['Exists?'] = 'Yes!!'
                               Yes.append((row['X'], row['Y']))
           #doing flux density
           for row in valid_rows_filtered:
               net_flx = row['Net Flux (Jy)']
               net_flx_err = row['Flux Uncertainty']
               #print(net_flx)
               flux_density, flux_density_unc = flux_dens(net_flx, net_flx_err, wavelength)
               row['Flux Density'].append(flux_density)
               row['Flux Density Uncertainty'].append(flux_density_unc)
                  
          
           # Update rows with valid_rows_filtered information
           for valid_row in valid_rows_filtered:
               for row in rows:
                   if row['band_id'] == valid_row['band_id'] and row['Region'] == valid_row['Region']:
                       row.update(valid_row)


           #print('valid sources', valid_rows_filtered)


           #print( len(detectedpos_all))
           if len(detectedpos_all) > 0:
               apertures_detected = CircularAperture(detectedpos_all, r=2)
           if len(Yes) > 0:
               apertures_Yes = CircularAperture(Yes, r=YesRadius)
          
       # xc = 244.422687    #19.014239
       # yc=  191.596758# 310.340772
      
                

           # Plotting for current image
           '''
           fig, ax = plt.subplots(subplot_kw={'projection': wcs})
           norm = ImageNormalize(cutout.data, stretch=SqrtStretch())
           for row in valid_rows_filtered: # use valid_rows_filtered to change it back to only the valid sources.
               target_aperture = CircularAperture((row['X'], row['Y']), row['Radius'])
               annulus_aperture = CircularAnnulus((row['X'], row['Y']), row['Annulus_Inner_Radius'], row['Annulus_Outer_Radius'])
               target_aperture.plot(color='red', lw=1.5, alpha=.5, ax=ax)
               annulus_aperture.plot(color='blue', lw=1.5, alpha=.5, ax=ax)
              
               apertures.plot(color='#98ff98', lw=.5, alpha=0.5)
               apertures_detected.plot(color='#FF4500', lw=.5, alpha=0.5)  #  #FF69B4 for hot pink
               apertures_Yes.plot(color='green', lw=.5, alpha=0.5)


               # curious ones
           # curious = CircularAperture((xc,yc),5)
           # curious.plot(color='red', lw=.5, alpha=0.5)
           ax.imshow(cutout.data, cmap='gray', norm=norm, interpolation='nearest')
           ax.set_xlabel('Right Ascension')
           ax.set_ylabel('Declination')
           plt.title(f'Band {band_labels[band_id]}')
           plt.show()
           yesses =[row for row in valid_rows_filtered if row['Exists?'] == 'Yes!!']
           '''
   else:
       print('nothing worked')
           


  


      
      






display_data = pd.DataFrame(rows)
#by_name = display_data.groupby('Galaxy Name')
#byname = by_name.apply(lambda x: x).reset_index(drop=True)
#print("\n FINAL Sources Grouped by Galaxy:")
#display(byname)
#byname_byband = by_name.groupby('band_id')
#display(byname_byband)


#pd.set_option("display.max_rows", None)
#pd.set_option("display.max_columns", None)
print('Number of valid sources: ',len(display_data.loc[display_data['Flag']== 'Valid']))
print('Number of sources that dont 100 percent overlap x4 (because it is iterating over all 4 bands and appending each one once): ',len(display_data))
print('Number of sources coincidental with WISE bright points within 5 arcsec: ',len(display_data.loc[display_data['Exists?']== 'Yes!!']))
#display(display_data.loc[display_data['Flag']== 'Valid'])
display(display_data.loc[display_data['Exists?']== 'Yes!!'])
#display(display_data)
#print(len(display_data.loc[display_data['Flag']== 'Valid']))






  
#'''














'''
























#define coordinates
ra= 359.45700000000016
dec= -32.592000000000013
pos = SkyCoord(ra=ra, dec=dec, unit= 'deg')
#print(pos)


# Lookup and define a service for ALLWISE Atlas images
# the website gave me the URL
allwise_service = vo.dal.SIAService("https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?")


#search the service for images covering within 1 arcsecond of the star. make this bigger if needed
im_table = allwise_service.search(pos=pos, size= 1*u.arcsec)
#im_table
im_table.to_table().colnames
#for i in range(len(im_table)):
   #print(im_table[i])




#group the x-ray sources for this galaxy. locate through merging
df1 = targetgals
df2 = g_huge


merged_data = pd.merge(df1, df2, left_on='source_id', right_on = 'Gname_Homogenized', how='inner')
columns = ['RA','Dec','Gname_Homogenized']
Xray_sources = merged_data[columns]
Galaxies = Xray_sources.groupby(['Gname_Homogenized'])


# just want NGC 7793s sources
sources_7793 = Xray_sources.loc[Xray_sources['Gname_Homogenized'] == 'NGC 7793']
#sources_7793
#define the X-ray sources of NGC 7793
ra = sources_7793['RA'].values
dec = sources_7793['Dec'].values


# defining a function to calculate the distances between two sources.
def dist(p1, p2):
   return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )


#defining a function that creates a circular mask around each source so that if something overlaps with it, that overlapping part is not included in the aperture photometry
def create_circular_mask(h,w,center,radius):
   Y, X = np.ogrid[:h, :w] # creating an open (more memory efficient) coordinate grid of the image
   dist_from_center = np.sqrt((X-center[0])**2+ (Y-center[1])**2)
   mask = dist_from_center <= radius # so that everything inside the radius receives a mask
   return mask


# define a mapping of the bands into labels to make it easier
band_labels = {'w1': 'W1', 'w2': 'W2', 'w3': 'W3', 'w4': 'W4'}
flux_zmfd = {'w1': 309.54 ,'w2': 171.787,'w3': 31.674,'w4': 8.363} # check if these worked by looking at the band 4 code above
instr_zpmag = {'w1': 20.73,'w2': 19.56,'w3': 17.6 ,'w4':12.98 }
wavelengths = {'w1': 3.4 ,'w2': 4.6,'w3': 12,'w4': 22}


#define function to get flux density per unit frequency (energy units)
def flux_dens(net_flx, net_flx_err, wavelength):
   flux_density = (net_flx * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   flux_density_unc = (net_flx_err * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   return flux_density, flux_density_unc


rows = []


#for galaxy in Galaxies:
  #  ra = galaxy['RA'].values
  #  dec = galaxy['Dec'].values
'''



/var/folders/nd/77pj43rx7wbcbp0wx2qjq2140000gp/T/ipykernel_38896/3567697035.py:45: DtypeWarning: Columns (533,534) have mixed types. Specify dtype option on import or set low_memory=False.
  huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')



Aligned target galaxies and grouped sources:
NGC 5128
<DALResultsTable length=4>
      sia_title        ...    coadd_id  
                       ...              
        object         ...     object   
---------------------- ... -------------
W4 Coadd 2022m425_ac51 ... 2022m425_ac51
W2 Coadd 2022m425_ac51 ... 2022m425_ac51
W1 Coadd 2022m425_ac51 ... 2022m425_ac51
W3 Coadd 2022m425_ac51 ... 2022m425_ac51
Galaxy: NGC 5128
RA values: [201.23369464 201.23657077 201.25128439 201.25416886 201.2610683
 201.26774089 201.26807596 201.26870325 201.26891047 201.27085808
 201.271053   201.27125626 201.27607209 201.27751756 201.2797316
 201.28065611 201.28167034 201.28392685 201.28742704 201.28818788
 201.28958782 201.29404139 201.29652584 201.29780373 201.29912637
 201.29988096 201.2999652  201.30128534 201.30361352 201.30796511
 201.30823944 201.30842718 201.31119065 201.31161746 201.31400316
 201.31528464 201.31563163 201.31557711 201.31668668 201.31790489
 201.31822799 201.31930379 201.31981

AttributeError: 'float' object has no attribute 'shape'